In [1]:
import os
import gc
import cv2
import openslide
import numpy as np
import pandas as pd
import keras

from PIL import Image
from sklearn.model_selection import StratifiedShuffleSplit
from keras import layers, models
from keras import backend as K
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from openslide.deepzoom import DeepZoomGenerator

from preprocessing import get_train_path
from preprocessing import open_slide, create_tiles
from preprocessing import create_patches
from model import create_model

Using TensorFlow backend.


## 1. Train Generator

In [2]:
# train.py
file_handles = []
def generator(samples,
              slide_paths,
              truth_paths,
              batch_size,
              patch_size=256,
              shuffle=True):
    
    slide0 = open_slide(slide_paths[0])
    slide1 = open_slide(slide_paths[1])
    slide2 = open_slide(slide_paths[2])
    slide3 = open_slide(slide_paths[3])

    file_handles.append(slide0)
    file_handles.append(slide1)
    file_handles.append(slide2)
    file_handles.append(slide3)

    # tiles
    tiles0 = create_tiles(slide0, tile_size=patch_size)
    tiles1 = create_tiles(slide1, tile_size=patch_size)
    tiles2 = create_tiles(slide2, tile_size=patch_size)
    tiles3 = create_tiles(slide3, tile_size=patch_size)

    start_x0, start_y0 = 0, 0
    start_x1, start_y1 = 0, 0
    start_x2, start_y2 = 0, 0
    start_x3, start_y3 = 0, 0
    if 'pos' in slide_paths[0]:
        start_x0 = int(slide0.properties.get('openslide.bounds-x', 0)) / patch_size
        start_y0 = int(slide0.properties.get('openslide.bounds-y', 0)) / patch_size
        truth0 = open_slide(truth_paths[0])
        truth_tiles0 = create_tiles(truth0, tile_size=16)
    
    if 'pos' in slide_paths[1]: 
        start_x1 = int(slide1.properties.get('openslide.bounds-x', 0)) / patch_size
        start_y1 = int(slide1.properties.get('openslide.bounds-y', 0)) / patch_size
        truth1 = open_slide(truth_paths[1])
        truth_tiles1 = create_tiles(truth1, tile_size=16)
        
    if 'pos' in slide_paths[2]:
        start_x2 = int(slide2.properties.get('openslide.bounds-x', 0)) / patch_size
        start_y2 = int(slide2.properties.get('openslide.bounds-y', 0)) / patch_size
        truth2 = open_slide(truth_paths[2])
        truth_tiles2 = create_tiles(truth2, tile_size=16)
        
    if 'pos' in slide_paths[3]:
        start_x3 = int(slide3.properties.get('openslide.bounds-x', 0)) / patch_size
        start_y3 = int(slide3.properties.get('openslide.bounds-y', 0)) / patch_size
        truth3 = open_slide(truth_paths[3])
        truth_tiles3 = create_tiles(truth3, tile_size=16)
        
    num_samples = len(samples)
    while 1:
        if shuffle:
            samples = samples.sample(frac=1)  # shuffling

        for offset in range(0, num_samples, batch_size):
            batch_samples = samples.iloc[offset:offset+batch_size]

            batch_tiles, batch_masks = [], []
            for slide_path, (y, x) in zip(batch_samples['slide_path'].values, 
                                          batch_samples['tile_loc'].values):
                
                mask_tile_zoom = np.zeros((patch_size,patch_size))
                if slide_path == slide_paths[0]:
                    img = tiles0.get_tile(tiles0.level_count-1, (x+start_x0, y+start_y0))
                    if 'pos' in slide_path:
                        mask_tile = truth_tiles0.get_tile(truth_tiles0.level_count-1, (x, y))
                        mask_tile = (cv2.cvtColor(np.array(mask_tile), cv2.COLOR_RGB2GRAY) > 0).astype(int)
                        # mask_size_up , 16 to 256
                        k, l = mask_tile.shape
                        for i in range(k):
                            for j in range(l):
                                for o in range(16):
                                    for p in range(16):
                                        mask_tile_zoom[i*16+o,j*16+p] = mask_tile[i][j]
                        
                elif slide_path == slide_paths[1]:
                    img = tiles1.get_tile(tiles1.level_count-1, (x+start_x1, y+start_y1))
                    if 'pos' in slide_path:
                        mask_tile = truth_tiles1.get_tile(truth_tiles1.level_count-1, (x, y))
                        mask_tile = (cv2.cvtColor(np.array(mask_tile), cv2.COLOR_RGB2GRAY) > 0).astype(int)
                        # mask_size_up , 16 to 256
                        k, l = mask_tile.shape
                        for i in range(k):
                            for j in range(l):
                                for o in range(16):
                                    for p in range(16):
                                        mask_tile_zoom[i*16+o,j*16+p] = mask_tile[i][j]
                
                elif slide_path == slide_paths[2]:
                    img = tiles2.get_tile(tiles2.level_count-1, (x+start_x2, y+start_y2))
                    if 'pos' in slide_path:
                        mask_tile = truth_tiles2.get_tile(truth_tiles2.level_count-1, (x, y))
                        mask_tile = (cv2.cvtColor(np.array(mask_tile), cv2.COLOR_RGB2GRAY) > 0).astype(int)
                        # mask_size_up , 16 to 256
                        k, l = mask_tile.shape
                        for i in range(k):
                            for j in range(l):
                                for o in range(16):
                                    for p in range(16):
                                        mask_tile_zoom[i*16+o,j*16+p] = mask_tile[i][j]

                elif slide_path == slide_paths[3]:
                    img = tiles3.get_tile(tiles3.level_count-1, (x+start_x3, y+start_y3))
                    if 'pos' in slide_path:
                        mask_tile = truth_tiles3.get_tile(truth_tiles3.level_count-1, (x, y))
                        mask_tile = (cv2.cvtColor(np.array(mask_tile), cv2.COLOR_RGB2GRAY) > 0).astype(int)
                        # mask_size_up , 16 to 256
                        k, l = mask_tile.shape
                        for i in range(k):
                            for j in range(l):
                                for o in range(16):
                                    for p in range(16):
                                        mask_tile_zoom[i*16+o,j*16+p] = mask_tile[i][j]

                
                if img.size != (patch_size, patch_size):
                    img = Image.new('RGB', (patch_size, patch_size))
                    mask_tile_zoom = np.zeros((patch_size, patch_size))
                    
                batch_tiles.append(np.array(img))
                batch_masks.append(mask_tile_zoom)
                
            # train_x & train_y
            train_x = np.array(batch_tiles)
            train_y = np.array(batch_masks)
            train_y = to_categorical(
                train_y, num_classes=2).reshape(train_y.shape[0], 256, 256, 2)
            
            train_x, train_y = next(
                ImageDataGenerator(rotation_range=90,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   brightness_range =(0.65, 1.)).flow(train_x, y=train_y, batch_size=batch_size))
            
            train_x /= 255.
            yield train_x, train_y

## 2. Model

In [3]:
model = create_model()

In [4]:
slide_path = '../data/train/image/positive/Slide003.mrxs'
truth_path = '../data/train/mask/positive/Slide003.png'

In [5]:
slide_paths = [slide_path] * 4
mask_paths =  [truth_path] *4

In [6]:
slide_paths, mask_paths = get_train_path()

In [7]:
slide_4_list_1 = [[55,55, 0, 0]]

In [8]:
columns = ['is_tissue','slide_path','is_tumor','is_all_tumor','tile_loc']

In [9]:
batch_size = 64
n_epochs = 1

In [10]:
for slides in slide_4_list_1:
    sample_group_df = pd.DataFrame(
            columns=['is_tissue','slide_path','is_tumor','is_all_tumor','tile_loc'])
    
    group_slide_path, group_mask_path = [], []
    for idx in slides:
        slide_path, truth_path = slide_paths[idx], mask_paths[idx]
        slide_path = '..' + slide_path
        truth_path = '..' + truth_path
        samples = create_patches(slide_path, truth_path)
        sample_group_df = sample_group_df.append(samples)
        group_slide_path.append(slide_path)
        group_mask_path.append(truth_path)
        
    num_samples = len(sample_group_df)
    if num_samples > 10000:
        num_samples = 10000
    samples = sample_group_df.sample(num_samples, random_state=42)
    samples.reset_index(drop=True, inplace=True)
    
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
    for train_index, test_index in split.split(samples, samples["is_tumor"]):
            train_samples = samples.loc[train_index]
            validation_samples = samples.loc[test_index]
            
    train_gen = generator(train_samples, group_slide_path, group_mask_path, batch_size)
    val_gen = generator(validation_samples, group_slide_path, group_mask_path, batch_size)
    
    model.fit_generator(train_gen, 
                        steps_per_epoch=np.ceil(len(train_samples)/batch_size),
                        epochs=n_epochs,
                        validation_data=val_gen,
                        validation_steps=np.ceil(len(validation_samples)/batch_size),
                        verbose=1)
    print('========= Model saving... =======')
#     model.save_weights('./model/unet.h5')
    print('========= Model saved!!!! ========')
    for fh in file_handles:
        fh.close()
    file_handles = []

    gc.collect()
    del train_gen
    del val_gen

/home/excelsiorcjh/D/dev/help/notebook/wide-res-net/preprocessing.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  samples['tile_loc'] = list(samples.index)


Epoch 1/1


ResourceExhaustedError: OOM when allocating tensor with shape[64,48,64,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node conv2d_15/convolution}} = Conv2D[T=DT_FLOAT, _class=["loc:@training/Adamax/gradients/batch_normalization_14/cond/Merge_grad/cond_grad"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](batch_normalization_14/cond/Merge, conv2d_15/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss/mul/_1287}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_10812_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU

In [23]:
model.save_weights('./model/incept-unet.h5')